<a href="https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U02/FraudDetection_imbalanced_classes_SOLUTION-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="Bilder/ost_logo.png" width="240" height="120" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2022 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>


# Imbalanced classification: Missbräuchliche Kreditkartenbezüge

Demonstration des Umgangs mit stark unausgewogenen Klassifikationsproblemen.

## Introduction

Dieses Beispiel befasst sich mit der
[Kaggle Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/)
Datensatz, um zu demonstrieren, wie
wie man ein Klassifizierungsmodell auf Daten mit stark unausgewogenen Klassen trainiert.

## Datenaufbereitung

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import csv
import numpy as np
import pandas as pd

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
#fname = "creditcard.csv"

fname='/content/gdrive/MyDrive/ANN_Data/creditcard.csv'


all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)


## (a) Aufteilen in einen Trainings- und Validierungsdatensatz

In [ ]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


## (b) Analyse des Klassenungleichgewichts in der Response

In [ ]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)Bes

## (c) Standardisierung der Daten auf Basis der Statistik der Trainingsdaten

In [ ]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std


## (d) Aufbauen eines binären Klassifizierungsmodells

In [ ]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()


## (e) Trainieren Sie das Modell mit dem `class_weight`-Argument 

In [ ]:
weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]
print(weight_for_0, weight_for_1)

In [ ]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

#callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

history=model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)


In [ ]:
history.history.keys()

## (f) Lernkurven

Plotten Sie die Lernkurven für folgende Metriken:

- `keras.metrics.FalseNegatives(name="fn")`
- `keras.metrics.FalsePositives(name="fp")`
- `keras.metrics.TrueNegatives(name="tn")`
- `keras.metrics.TruePositives(name="tp")`
- `keras.metrics.Precision(name="precision")`
- `keras.metrics.Recall(name="recall")`



In [ ]:
import matplotlib.pyplot as plt

def plot_history(history, keyList):
    plt.figure()

    for key in keyList:
        val_key='val_'+key
        plt.plot(history.history[key])
        plt.plot(history.history[val_key])
        plt.title(key)
        plt.ylabel(key)
        plt.xlabel('epoch')
        plt.legend([key, val_key], loc='upper left')
        plt.grid(True); plt.show()
    

In [ ]:
plot_history(history,['loss','precision','recall','tp','tn'])

## (g) Interpretation, Bewertung des Learners

In [ ]:
history.history['val_tp'][-1]

In [ ]:
history.history['val_fp'][-1]

In [ ]:
history.history['val_fn'][-1]

Am Ende des Trainings haben wir von 56.961 Validierungstransaktionen:

- 66 von ihnen korrekt als betrügerisch identifiziert
- 9 betrügerische Transaktionen werden übersehen
- um den Preis, dass wir 441 legitime Transaktionen falsch kennzeichnen

In der realen Welt würde man der Klasse 1 ein noch höheres Gewicht beimessen,
um der Tatsache Rechnung zu tragen, dass falsche Negativmeldungen teurer sind als falsche Positivmeldungen.

Das nächste Mal, wenn Ihre Kreditkarte bei einem Online-Einkauf abgelehnt wird - das ist der Grund.

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)